#Biblioteca de Dados


https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features




* `Acousticness/Acústica:` Variável numérica, medida de confiança de 0,0 a 1,0 se a faixa é acústica. 1.0 representa alta confiança de que a faixa é acústica.

* `Danceability/Dançabilidade:` Variável numérica, a dançabilidade descreve o quão adequada uma faixa é para dançar com base em uma combinação de elementos musicais, incluindo tempo, estabilidade do ritmo, força da batida e regularidade geral. Um valor de 0,0 é o menos dançável e 1,0 é o mais dançável.

* `Duration_ms:`Variável numérica, a duração da trilha em milissegundos.

* `Duration_min:` Variável numérica, a duração da faixa em minutos.

* `Energy/Energia:` Variável numérica, Energia é uma medida de 0,0 a 1,0 e representa uma medida perceptiva de intensidade e atividade. Normalmente, as faixas energéticas parecem rápidas, altas e barulhentas. Por exemplo, o death metal tem alta energia, enquanto um prelúdio de Bach tem uma pontuação baixa na escala. As características perceptivas que contribuem para este atributo incluem faixa dinâmica, intensidade percebida, timbre, taxa de início e entropia geral.

* `Explicit/Explícito:` Variável categórica, se a faixa tem ou não letras explícitas (verdadeiro = sim (1); falso = não(0), não OU desconhecido).

* `Id:` O ID do Spotify para a faixa.

* `Instrumentalness/Instrumentalidade:` Variável numérica, prevê se uma faixa não contém vocais. Os sons “Ooh” e “aah” são tratados como instrumentais neste contexto. Faixas de rap ou de palavras faladas são claramente “vocais”. Quanto mais próximo o valor de instrumentalidade estiver de 1,0, maior a probabilidade de a faixa não conter conteúdo vocal. Valores acima de 0,5 destinam-se a representar faixas instrumentais, mas a confiança é maior à medida que o valor se aproxima de 1,0.

* `Key/Chave:`Variável numérica, a chave geral estimada da faixa. Os inteiros são mapeados para pitchs usando a notação padrão de Pitch Class. Por exemplo. 0 = C, 1 = C#/Db, 2 = D, e assim por diante. Se nenhuma chave foi detectada, o valor é -1.

* `Liveness/ Ao vivo:` Variável numérica, detecta a presença de um público na gravação. Valores mais altos de vivacidade representam uma probabilidade maior de que a faixa tenha sido executada ao vivo. Um valor acima de 0,8 fornece uma forte probabilidade de que a faixa esteja ativa.

* `Loudness/ Volume em dB:` Variável numérica, volume geral de uma faixa em decibéis (dB). Os valores de volume são calculados em média em toda a faixa e são úteis para comparar o volume relativo das faixas. A sonoridade é a qualidade de um som que é o principal correlato psicológico da força física (amplitude). Os valores típicos variam entre -60 e 0 db.

* `Mode/ Modo:` Variável numérica, o modo indica a modalidade (maior ou menor) de uma faixa, o tipo de escala da qual seu conteúdo melódico é derivado. Maior é representado por 1 e menor é 0.

* `Popularity/Popularidade:` Variável numérica, a popularidade de uma faixa é um valor entre 0 e 100, sendo 100 o mais popular. A popularidade é calculada por algoritmo e é baseada, em grande parte, no número total de execuções que a faixa teve e quão recentes são essas execuções.

* `Speechiness/Fala:` Variável numérica, a fala detecta a presença de palavras faladas em uma faixa. Quanto mais exclusivamente falada a gravação (por exemplo, talk show, audiolivro, poesia), mais próximo de 1,0 o valor do atributo. Valores acima de 0,66 descrevem faixas que provavelmente são feitas inteiramente de palavras faladas. Valores entre 0,33 e 0,66 descrevem faixas que podem conter música e fala, seja em seções ou em camadas, incluindo casos como música rap. Os valores abaixo de 0,33 provavelmente representam músicas e outras faixas que não são de fala.

* `Tempo:` Variável numérica, Tempo estimado geral de uma faixa em batidas por minuto (BPM). Na terminologia musical, tempo é a velocidade ou ritmo de uma determinada peça e deriva diretamente da duração média da batida.

* `Valence/Valência:` Variável numérica, Medida de 0,0 a 1,0 descrevendo a positividade musical transmitida por uma faixa. Faixas com alta valência soam mais positivas (por exemplo, feliz, alegre, eufórica), enquanto faixas com baixa valência soam mais negativas (por exemplo, triste, deprimida, irritada).

* `Year/Ano:` Ano em que a música foi lançada.

#Instalando o PySpark


In [ ]:
!pip install pyspark

In [ ]:
#criando a SparkSession

from pyspark.sql import SparkSession

sessao_spark = SparkSession\
                    .builder\
                    .appName('Recomendador PySpark')\
                    .getOrCreate()

sessao_spark

#Leitura dos dados

In [ ]:
from pyspark import SparkFiles

In [ ]:
url_dados = 'dados_musicas.csv'

In [ ]:
#adiciona a url à sessao spark e depois pega caminho no contexto spark e concatena com o nome do arquivo

sessao_spark.sparkContext.addFile(url_dados)
path_dados_file = 'file://' + SparkFiles.get('dados_musicas.csv')

#le o csv, mantendo o header e deixando o schema automático
dados = sessao_spark.read.csv(path_dados_file, header=True, sep=';', inferSchema=True)

dados.show()

In [ ]:
#mostra o schema dos dados
dados.printSchema()

In [ ]:
#conta as linhas
dados.count()

In [ ]:
#conta as colunas
len(dados.columns)

In [ ]:
import pyspark.sql.functions as f

In [ ]:
#para cada c em colunas, conte, quando, é nulo(c), se a condição for verdadeira retorne 1 para contar o total de valores nulos

dados.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in dados.columns]).show()

In [ ]:
#oredena a coluna years
print(sorted(dados.select('year').distinct().collect()))

#Segunda fonte de dados

In [ ]:
#carrega csv de anos
url_anos_dados = 'dados_musicas_ano.csv'

#add à sessão spark, mantendo o header e prevendo o schema
sessao_spark.sparkContext.addFile(url_anos_dados)
path_dados_file = 'file://' + SparkFiles.get('dados_musicas_ano.csv')
dados_anos = sessao_spark.read.csv(path_dados_file, header=True, inferSchema=True)
dados_anos.show(100)


In [ ]:
#dados sobre anos filtrados em maior que 2000
dados_anos = dados_anos.filter('year >= 2000')
dados_anos.show(100)

In [ ]:
#quantidade de linhas
dados_anos.count()

In [ ]:
#quantidade de colunas
len(dados_anos.columns)

In [ ]:
import plotly.express as px

In [ ]:
#plot do gráfico mostrando a variação de loudness vs anos
fig = px.line(dados_anos.toPandas(), x='year', y='loudness', markers=True, title='Variação do loudness conforme os anos')
fig.show()

In [ ]:
import plotly.graph_objects as go

In [ ]:
#converte um dataframe spark em dataframe pandas
#cria uma figura vazia para criar um grafico acousticness vs year
temp = dados_anos.toPandas()
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['year'], y=temp['acousticness'], name='Acousticness'))
fig.show()

In [ ]:
#transforma o dataframe em pandas e cria um gráfico interativo para várias métricas musicais

import plotly.graph_objects as go

temp = dados_anos.toPandas()

fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['year'], y=temp['acousticness'], name='Acousticness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['valence'], name='Valence'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['danceability'], name='Danceability'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['energy'], name='Energy'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['instrumentalness'], name='Instrumentalness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['liveness'], name='Liveness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['speechiness'], name='Speechiness'))

fig.show()


#Matriz de correlação

In [ ]:
#transforma em pandas e cria um heatmap interativo do plotly
fig = px.imshow(dados_anos.drop('mode').toPandas().corr(), text_auto=True)
fig.show()


#Clusterização por gênero

**PCA e StandardScaler**

In [ ]:
#carrega csv de genero
url_dados_generos = 'dados_musicas_genero.csv'

#add o csv à sessão spark mantendo o header e prevendo o schema
sessao_spark.sparkContext.addFile(url_dados_generos)
path_dados_file = 'file://' + SparkFiles.get('dados_musicas_genero.csv')
dados_generos = sessao_spark.read.csv(path_dados_file, header=True, inferSchema=True)
dados_generos.show()

In [ ]:
#quantidade de linhas
dados_generos.count()

In [ ]:
#quantidade de linhas distintas
dados_generos.select('genres').distinct().count()

In [ ]:
#quantidade de colunas
len(dados_generos.columns)

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
#mostra todas as colunas
dados_generos.columns

In [ ]:
#exclui o genero
x = dados_generos.columns
x.remove('genres')
x

In [ ]:
#transforma todas as colunas numéricas em vetor, criando uma única coluna de features
dados_generos_vector = VectorAssembler(inputCols=x, outputCol='features').transform(dados_generos).select(['features', 'genres'])
dados_generos_vector.show(truncate=False, n=5)

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
#faz a normalização das features para que tenham a mesma escala antes de treinar o modelo
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(dados_generos_vector)
dados_generos_scaler = scaler_model.transform(dados_generos_vector)
dados_generos_scaler.show()

In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
#PCA = Análise de Componentes Principais
#reduz a dimensionalidade das features, reduz as variáveis para 2 componentes principais
pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_generos_scaler)
dados_generos_pca = model_pca.transform(dados_generos_scaler)
dados_generos_pca.select('pca_features').show(truncate=False)

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
#Aplica o pipeline treinado sobre dados_generos
#evita ter que executar cada etapa manualmente
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=x, outputCol='features'),
                                StandardScaler(inputCol='features', outputCol='scaled_features'),
                                PCA(k=2, inputCol='scaled_features', outputCol='pca_features')])
pca_pipeline_model = pca_pipeline.fit(dados_generos)
dados_generos_pca = pca_pipeline_model.transform(dados_generos)
dados_generos_pca.show()


#K-means

In [ ]:
from pyspark.ml.clustering import KMeans

In [ ]:
#aplica o algoritmo KMeans nos dados reduzidos pelo PCA, dividindo em 5 clusters
#a nova coluna 'cluster_pca mostra a qual grupo cada linha pertence.
kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca').setK(5).setSeed(1224)
model_kmeans = kmeans.fit(dados_generos_pca)
predictions_kmeans = model_kmeans.transform(dados_generos_pca)
predictions_kmeans.select('pca_features', 'cluster_pca').show(truncate=False)

**Plotando o cluster**

In [ ]:
from pyspark.ml.functions import vector_to_array

In [ ]:
#converte o vetor pca_features em duas colunas separadas, adiciona o cluster atribuído e o gênero original para facilitar a visualização
pca_features_xy = predictions_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
                                    .withColumn('y', vector_to_array('pca_features')[1])\
                                    .select(['x', 'y', 'cluster_pca', 'genres'])
pca_features_xy.show()

In [ ]:
#cria um scatter plot colorindo por cluster atribuído pelo KMeans
fig = px.scatter(pca_features_xy.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['x', 'y', 'genres'])
fig.show()

In [ ]:
#mostra a variância ecplicada pelo PCA indicando quanto da informação original foi preservada
pca_pipeline_model.stages[2].explainedVariance

In [ ]:
# Na definição da função vamos incluir os parâmetros que gostariamos de variar, como quantidade componentes, número de clusters e colunas utilizadas
def cria_pipeline(k_PCA=2, SEED=1224, k_kmeans=5, columns=x, data=dados_generos):


  # Definição do Pipeline
  # Primeiro a vetorização das colunas selecionadas
  # Segundo a padronização dos dados com o StandardScaler
  # Terceiro o PCA com o número k_PCA de componentes
  # Quarto o KMeans com o número k_kmeans de clusters
  pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols = columns, outputCol = 'features'),\
                                StandardScaler(inputCol='features', outputCol='scaled_features'),\
                                PCA(k=k_PCA, inputCol="scaled_features", outputCol='pca_features'),\
                                KMeans(featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED, k=k_kmeans)])

  # ajuste da pipeline com os dados
  model = pca_pipeline.fit(data)

  # transformação dos dados utilizando o modelo da pipeline
  predictions_kmeans = model.transform(data)

  # criando duas novas colunas a partir das primeiras duas componentes do PCA
  pca_features_xs = predictions_kmeans.withColumn("x", vector_to_array("pca_features")[0])\
                   .withColumn("y", vector_to_array("pca_features")[1])\
                   .select(['x', 'y', 'cluster_pca', 'genres'])

  # plotando as duas componentes e o cluster
  fig = px.scatter(
   pca_features_xs.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['x', 'y', 'genres'])
  fig.show()

  # retornando pipeline ajustado
  return model

#Clusterização por Música

**Redução de dimensionalidade com PCA**

In [ ]:
dados.show()

In [ ]:
#remoção das colunas não numéricas
x = dados.columns
x.remove('artists')
x.remove('id')
x.remove('name')
x.remove('artists_song')
x

In [ ]:
#vetoriza as colunas numéricas na coluna features, preparando para modelos de ML
dados_encoded_vector = VectorAssembler(inputCols=x, outputCol='features').transform(dados)
dados_encoded_vector.select('features').show(truncate=False, n=5)

In [ ]:
#normalização dos vetores para que todos fiquem na mesma escala
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')
model_scaler = scaler.fit(dados_encoded_vector)
dados_musicas_scaler = model_scaler.transform(dados_encoded_vector)
dados_musicas_scaler.select('features_scaled').show(truncate=False, n=5)

In [ ]:
#quantidade de colunas de x
k = len(x)
k

In [ ]:
#aplica a PCA para diminuir a dimensionalidade das features ja normalizadas com K componentes principais
pca = PCA(k=k, inputCol='features_scaled', outputCol='pca_features')
model_pca = pca.fit(dados_musicas_scaler)
dados_musica_pca = model_pca.transform(dados_musicas_scaler)
sum(model_pca.explainedVariance) * 100

In [ ]:
#cria uma lista com a variância acumulada explicada até cada componente principal
lista_valores = [sum(model_pca.explainedVariance[0:i+1]) for i in range(k)]
lista_valores

In [ ]:
import numpy as np

In [ ]:

#define k como o número de componentes principais necessários para explicar pelo menos 70% da variância dos dados
k = sum(np.array(lista_valores) <= 0.7)
k

In [ ]:
#aplica o PCA usando o número ótimo de componentes (k) reduzindo as 'features_scaled' para 'pca_features' e preservando pelo menos 70% da variância dos dados
pca = PCA(k=k, inputCol='features_scaled', outputCol='pca_features')
model_pca = pca.fit(dados_musicas_scaler)
dados_musicas_pca_final = model_pca.transform(dados_musicas_scaler)
dados_musicas_pca_final.select('pca_features').show(truncate=False)

In [ ]:
#mostra a porcentagem da variância total explicada pelos k componentes principais escolhidos
sum(model_pca.explainedVariance) * 100

**Aplicação do cluster com K-means**

In [ ]:
#cria um pipeline com VectorAssembler, StandardScaler e PCA (k=15), gera a projeção dos dados em 15 componentes principais e exibe os primeiros resultados
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=x, outputCol='features'),
                        StandardScaler(inputCol='features', outputCol='featured_scaled'),
                        PCA(k=15, inputCol='featured_scaled', outputCol='pca_features')])
model_pca_pipeline = pca_pipeline.fit(dados)
projection = model_pca_pipeline.transform(dados)
projection.select('pca_features').show(truncate=False, n=5)

In [ ]:
#aplica KMeans com k=50 usando os 'pca_features' como entrada. A coluna 'cluster_pca' indica o número do cluster atribuído a cada ponto
kmeans = KMeans(k=50, featuresCol='pca_features', predictionCol='cluster_pca', seed=1224)
modelo_kmeans = kmeans.fit(projection)
projection_kmeans = modelo_kmeans.transform(projection)
projection_kmeans.select('pca_features', 'cluster_pca').show()

In [ ]:
#extrai os dois primeiros componentes principais (x e y) de 'pca_features', junto com o cluster atribuído e o nome da música
projection_kmeans = projection_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
                                      .withColumn('y', vector_to_array('pca_features')[1])
projection_kmeans.select(['x', 'y', 'cluster_pca', 'Artists_song']).show()

**Analisando o cluster**

In [ ]:
#cria um scatter usando os dois primeiros componentes principais (x e y), colorindo os pontos conforme o cluster e mostrando o nome da música no hover.
fig = px.scatter(projection_kmeans.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['artists_song'])
fig.show()

In [ ]:
nome_musica = 'Avenged Sevenfold - Nightmare'


In [ ]:
#filtra a música pelo nome e mostra o número do cluster em  ela foi colocada
cluster = projection_kmeans.filter(projection_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
cluster


In [ ]:
#seleciona todas as músicas que pertencem ao mesmo cluster da música escolhida, retornando o nome do artista/música, o id e suas features em PCA.
musicas_recomendadas = projection_kmeans.filter(projection_kmeans.cluster_pca == cluster).select('artists_song', 'id', 'pca_features')
musicas_recomendadas.show()

In [ ]:
#obtém o vetor da pca_features da música escolhida, que representa sua posição no espaço reduzido do PCA.
componentes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica).select('pca_features').collect()[0][0]
componentes_musica

In [ ]:
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType

In [ ]:
#cria uma UDF para calcular a distância euclidiana entre a música escolhida e as músicas do mesmo cluster, ordena pela menor distância e retorna as 10 músicas mais próximas
def calcula_distance(value):
  return euclidean(componentes_musica, value)


udf_calcula_distance = f.udf(calcula_distance, FloatType())

musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

recomendadas = sessao_spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

recomendadas.show()

In [ ]:
# Dado o nome de uma música:
# 1) Descobre o cluster dela (KMeans em PCA).
# 2) Seleciona todas as músicas do mesmo cluster.
# 3) Calcula a distância euclidiana no espaço PCA até a música escolhida.
# 4) Ordena por distância e mostra as 10 mais parecidas (recomendação)



from os import truncate
def recomendador(nome_musica):
  cluster = projection_kmeans.filter(projection_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
  musicas_recomendadas = projection_kmeans.filter(projection_kmeans.cluster_pca == cluster).select('artists_song', 'id', 'pca_features')
  componentes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica).select('pca_features').collect()[0][0]

  def calcula_distance(value):
    return euclidean(componentes_musica, value)


  udf_calcula_distance = f.udf(calcula_distance, FloatType())

  musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

  recomendadas = sessao_spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

  recomendadas.select('artists_song').show(truncate=False)

In [ ]:
recomendador('Avenged Sevenfold - Nightmare')

In [ ]:
#retorna True se encontrou a música no dataframe, e False se não encontrou
def encontra_musica(nome_musica, predictions_kmeans):
  resultado = predictions_kmeans.filter(predictions_kmeans.artists_song == nome_musica).collect()
  if len(resultado) == 0:
    return False
  return True

#Biblioteca Spotipy

In [ ]:
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentialsfrom google.colab import userdata

In [ ]:
#autenticação da biblioteca do spotipy
scope = 'user-library-read playlist-modify-private'

OAuth = SpotifyOAuth(
    scope=scope,
    redirect_uri='http://127.0.0.1:8888/callback',
    client_id='1a74a1bbfdea476f9ee370ee252fd01a',
    client_secret=userdata.get('secret_spotify')
)

In [ ]:
#credenciais para usar o spotipy
client_credentials_manager = SpotifyClientCredentials(client_id='1a74a1bbfdea476f9ee370ee252fd01a',
                                                      client_secret=userdata.get('secret_spotify'))

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
#recupera o ID da música escolhida a partir do nome
id = projection_kmeans.filter(projection_kmeans.artists_song == nome_musica).select('id').collect()[0][0]
id

In [ ]:
sp.track(id)

In [ ]:
#calcula músicas recomendadas, Descobre o cluster da música-alvo, Seleciona todas as músicas do mesmo cluster, Pega os componentes PCA da música-alvo
def recomendador(nome_musica):

  cluster = projection_kmeans.filter(projection_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
  musicas_recomendadas = projection_kmeans.filter(projection_kmeans.cluster_pca == cluster).select('artists_song', 'id', 'pca_features')
  componentes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica).select('pca_features').collect()[0][0]

  #Função para calcular distância euclidiana entre músicas
  def calcula_distance(value):
    return euclidean(componentes_musica, value)


  udf_calcula_distance = f.udf(calcula_distance, FloatType())

  musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

  recomendadas = sessao_spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

  recomendadas.select('artists_song').show(truncate=False)


  #pegar informações da api
  playlist_id = recomendadas.select('id').collect()
  playlist_track = []

  for id in playlist_id:
    playlist_track.append(sp.track(id[0]))

  return len(playlist_track)


In [ ]:
recomendador('Avenged Sevenfold - Nightmare')

#Imagem do álbum

In [ ]:
import matplotlib.pyplot as plt
from skimage import io

In [ ]:
#Consulta na API os detalhes da música, pega a URL da imagem, plota a capa do álbum

nome_musica = 'Avenged Sevenfold - Nightmare'

id = projection_kmeans.filter(projection_kmeans.artists_song == nome_musica)\
                 .select('id').collect()[0][0]
track = sp.track(id)
url = track["album"]["images"][1]["url"]
name = track["name"]

image = io.imread(url)
plt.imshow(image)
plt.xlabel(name, fontsize = 10)
plt.show()

In [ ]:
def visualize_songs(name, url):
    plt.figure(figsize=(15, 10))
    columns = 5
    for i, u in enumerate(url):
        ax = plt.subplot(len(url) // columns + 1, columns, i + 1)
        image = io.imread(u)
        ax.imshow(image)

        # Remove eixos, ticks e bordas
        ax.axis('off')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_frame_on(False)

        # Nome da música embaixo da imagem
        ax.text(
            0.5, -0.05, name[i],
            fontsize=9, ha='center', va='top', wrap=True,
            transform=ax.transAxes
        )

    # Cola as capas com mínimo espaço, mas deixa espaço pro nome
    plt.tight_layout(pad=0)
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show()


In [ ]:
#pega os IDs, cria listas vazias para guardar nome e URL das capas, consulta na API, chama a função para visualizar as capas
playlist_id = recomendadas.select('id').collect()

name = []
url = []

for i in playlist_id:
  track = sp.track(i[0])
  url.append(track['album']['images'][1]['url'])
  name.append(track['name'])

visualize_songs(name,url)

In [ ]:
def recomendador(nome_musica):
  #calcula músicas recomendadas
  cluster = projection_kmeans.filter(projection_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
  musicas_recomendadas = projection_kmeans.filter(projection_kmeans.cluster_pca == cluster).select('artists_song', 'id', 'pca_features')
  componentes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica).select('pca_features').collect()[0][0]

  #calcula a distância euclidiana para ver as músicas mais próximas
  def calcula_distance(value):
    return euclidean(componentes_musica, value)


  udf_calcula_distance = f.udf(calcula_distance, FloatType())

  musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

  recomendadas = sessao_spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])


  #pegar informações da api
  playlist_id = recomendadas.select('id').collect()

  name = []
  url = []

  for i in playlist_id:
    track = sp.track(i[0])
    url.append(track['album']['images'][1]['url'])
    name.append(track['name'])


  #plotando capas
  plt.figure(figsize=(15, 10))
  columns = 5
  for i, u in enumerate(url):
      ax = plt.subplot(len(url) // columns + 1, columns, i + 1)
      image = io.imread(u)
      ax.imshow(image)

      ax.axis('off')
      ax.set_xticks([])
      ax.set_yticks([])
      ax.set_frame_on(False)

      ax.text(
          0.5, -0.05, name[i],
          fontsize=9, ha='center', va='top', wrap=True,
          transform=ax.transAxes
      )

  plt.tight_layout(pad=0)
  plt.subplots_adjust(wspace=0.02, hspace=0.02)
  plt.show()



In [ ]:
recomendador('Avenged Sevenfold - Nightmare')